# Обзор способов решения

Самым простым и очевидным способом постановки задачи (как и многих проблем в NLP) является tokens classification. Решать задачу можно с помощью марковских моделей (HMM, CRF), рекуррентных нейронных сетей, сверточных нейронных сетей, архитектуры Transformer. В данной работе рассматривается только последняя. Конечно же, веса инициализируются весами модели с архитектурой BERT, предобученной на большом корпусе русского языка (DeepPavlov/rubert-base-cased). К эмбеддингам с последнего скрытого слоя на вход классификатору можно добавить POS - фичи, но на это не хватило времени. К тому же, в https://www.hse.ru/en/edu/vkr/296279742 показывается, что при использовании character-level информации такие фичи не нужны.

Если поискать другие решения задачи, то можно найти основанное на построении синтаксического дерева. 
https://arxiv.org/pdf/1906.11298.pdf , 
https://www.researchgate.net/publication/270878718_Punctuation_Prediction_with_Transition-based_Parsing . Для применения таких методов нужно построить синтаксическое дерево. Модели для построения таких деревьев по тексту на русском языке не содержатся в пакетах nltk и spacy, однако можно найти предобученную или обучить самостоятельно. Ввиду связанных с этим дополнительных трудозатрат, такой подход не был применен в этой работе.

# Описание решения



Для обучения используется новости с сайта Lenta из корпуса "Tayga", раздел "All News" (https://tatianashavrina.github.io/taiga_site/downloads). В корпусе уже расставлена пунктуация, ее только необходимо выделить. Набор знаков ограничен теми, которую я сам считаю пунктуацией: **!,-.:;?()**. 

Каждый текст сначала разбивается на предложения с помощью nltk.wordpunct_tokenize. Каждый сэмпл состоит из трех предложений. Затем сэмпл разбивается на токены. Каждый токен, являющийся пунктуацией, удаляется и "приклеивается" в качестве класса к предыдущему непунктуационному токену. Таким образом, получается датасет для классификации токенов.

Для обучение модели на основе архитектуры Transformers используется библиотека HuggingFace Transformers. Модель строится следующим образом:


```python
import transformers
from utils import PUNCT_TO_ID

config = transformers.AutoConfig.from_pretrained('DeepPavlov/rubert-base-cased')
config.num_labels = len(PUNCT_TO_ID) # Поменять количество выходов на слое классификации
model = transformers.AutoModelForTokenClassification.from_pretrained('DeepPavlov/rubert-base-cased', config=config)
```



Код доступен в репозитории https://github.com/Lesha17/Punctuation.git

# Проверка результатов

In [1]:
!pip install transformers

     |████████████████████████████████| 778kB 6.3MB/s 
     |████████████████████████████████| 3.0MB 18.2MB/s 
     |████████████████████████████████| 890kB 35.1MB/s 
     |████████████████████████████████| 1.1MB 37.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e9af8c1fa7890bd7a70bfcd1a60e308d1df46177bd7f8e38d7488a66f627a679
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!git clone https://github.com/Lesha17/Punctuation.git

Cloning into 'Punctuation'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 18 (delta 5), reused 17 (delta 4), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [9]:
!mkdir Lenta
!unzip -q Punctuation/data/Lenta_split.zip -d Lenta

mkdir: cannot create directory ‘Lenta’: File exists


In [3]:
!unzip model.zip -d model

Archive:  model.zip
  inflating: model/config.json       
  inflating: model/pytorch_model.bin  


In [1]:
import transformers

In [2]:
model = transformers.AutoModelForTokenClassification.from_pretrained('model').to('cuda')

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

In [ ]:
import nltk
nltk.download('punkt')

In [4]:
import sys
sys.path.append('Punctuation')

In [5]:
from data_reader import PunctuationDataset
from utils import PUNCT_TO_ID
import torch
import tqdm
import numpy as np

In [6]:
BATCH_SIZE = 128

def read(dataset):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=transformers.default_data_collator)
    model.eval()
    true_labels = []
    outputs = []
    mask = []
    for batch in tqdm.autonotebook.tqdm(dataloader):
        torch.cuda.empty_cache()
        batch = {k: t.to('cuda') for k, t in batch.items()}
        with torch.no_grad():
            model_outputs = model(**batch)
        true_labels.append(batch['labels'].cpu())
        outputs.append(model_outputs[1].cpu())
        mask.append(batch['attention_mask'].cpu())

    true_labels = np.concatenate(true_labels, axis=0)
    outputs = np.concatenate(outputs, axis=0)
    mask = np.concatenate(mask, axis=0)

    outputs = outputs.reshape(-1, 10)
    true_labels = true_labels.reshape(-1)
    mask = mask.reshape(-1)
    outputs = outputs[mask != 0]
    true_labels = true_labels[mask != 0]
    
    return outputs, true_labels

In [7]:
from sklearn.metrics import accuracy_score

def custom_acc(y_true, y_pred):
    interest_idx = np.logical_or(y_pred != 0, y_true != 0)
    return accuracy_score(y_true[interest_idx], y_pred[interest_idx])

In [8]:
dataset_dev = PunctuationDataset(data_dir='Lenta/dev', tokenizer=tokenizer, label_to_idx=PUNCT_TO_ID, batch_size=BATCH_SIZE)
dev_outputs, dev_true_labels = read(dataset_dev)

Caclulating length of dataset Lenta/dev


In [9]:
dataset_test = PunctuationDataset(data_dir='Lenta/test', tokenizer=tokenizer, label_to_idx=PUNCT_TO_ID, batch_size=BATCH_SIZE)
test_outputs, test_true_labels = read(dataset_test)

Caclulating length of dataset Lenta/test


In [10]:
dev_labels_predict = np.argmax(dev_outputs, axis=-1)
test_labels_predict = np.argmax(test_outputs, axis=-1)

In [11]:
custom_acc(dev_true_labels, dev_labels_predict)

0.8976816089587333

In [12]:
custom_acc(test_true_labels, test_labels_predict)

0.895375187248638

И теперь применяем модель

In [13]:
from utils import PUNCTUATION, PUNCT_TO_ID

def restore_punct(token_ids, tokenizer, token_label_ids):
    result = ''
    for token_id, label_id in list(zip(token_ids, token_label_ids)):
        token_id = token_id.item()
        if token_id == tokenizer.cls_token_id:
            continue
        if token_id in (tokenizer.sep_token_id, tokenizer.pad_token_id):
            break
        token = tokenizer.ids_to_tokens[token_id]
        if token.startswith('##'):
            result = result[:-2] # remove last added punctuation
            token = token[2:]
        elif len(result) > 0 and result[-1] != ' ':
            result += ' '
            
        result += token
        result += ' ' + (' ' + PUNCTUATION)[label_id]
    return result

def make_punct(texts, model, tokenizer):
    encoded = tokenizer(texts, max_length=192, padding="max_length", truncation=True, return_tensors='pt')
    encoded = {k: v.cuda() for k, v in encoded.items()}
    with torch.no_grad():
        model_out = model(**encoded)
    predicted_tokens = torch.argmax(model_out[0], dim=-1)
    results = []
    for sample_ids, sample_predicts in list(zip(encoded['input_ids'], predicted_tokens)):
        result = restore_punct(sample_ids, tokenizer, sample_predicts)

        results.append(result)
    return results

In [14]:
make_punct(["Начиная жизнеописание героя моего Алексея Федоровича Карамазова нахожусь в некотором недоумении" \
           "А именно хотя я и называю Алексея Федоровича моим героем но однако сам знаю что человек он " \
           "отнюдь не великий а посему и предвижу неизбежные вопросы вроде таковых чем же замечателен ваш " \
           "Алексей Федорович что вы выбрали его своим героем"], model, tokenizer)

['начиная  жизнеописание  героя  моего , алексея  федоровича , карамазова  нахожусь  в  некотором  недоуменииа . именно  хотя  я  и  называю  алексея  федоровича  моим  героем , но  однако  сам  знаю , что  человек  он  отнюдь  не  великии , а  посему  и  предвижу  неизбежные  вопросы  вроде  таковых . чем  же  замечателен  ваш  алексеи  федорович , что  вы  выбрали  его  своим  героем ?']

In [15]:
from nltk.tokenize import wordpunct_tokenize

def check_one(reference, hypothesis):
    correct = 0
    incorrect = 0
    ref = wordpunct_tokenize(reference)
    hyp = wordpunct_tokenize(hypothesis)
    ref_i, hyp_i = 0, 0
    punct_places = 0
    while ref_i < len(ref) and hyp_i < len(hyp):
        need_punct_check_ref = False
        need_punct_check_hyp = False
        cur_ref = ref[ref_i]
        if cur_ref in PUNCT_TO_ID:
            need_punct_check_ref = True
            punct_places += 1
        cur_hyp = hyp[hyp_i]
        if cur_hyp in PUNCT_TO_ID:
            need_punct_check_hyp = True
        if need_punct_check_ref and need_punct_check_hyp:
            if cur_ref == cur_hyp:
                correct += 1
            else:
                incorrect += 1
            ref_i += 1
            hyp_i += 1
            continue

        if need_punct_check_ref and not need_punct_check_hyp:
            incorrect += 1
            ref_i += 1
            continue

        if not need_punct_check_ref and need_punct_check_hyp:
            incorrect += 1
            hyp_i += 1
            continue

        assert cur_hyp == cur_ref, "The phrases are inconsistent!" + cur_hyp + ' ' + cur_ref
        ref_i += 1
        hyp_i += 1
    if punct_places == 0:
        return 1 - incorrect/(2 * len(reference))
        
    return correct/punct_places - incorrect/(2 * len(reference))

In [16]:
def prepare(s, keep_punct=True):
    encoded = tokenizer(s, max_length=192, truncation=True, add_special_tokens=False)
    result = ''
    for idx in encoded.input_ids:
        token = tokenizer.ids_to_tokens[idx]
        if keep_punct or token not in PUNCT_TO_ID:
            result += token + ' '
    return result.replace(' ##', '')

In [17]:
from nltk import sent_tokenize, wordpunct_tokenize
import os

def eval_data(data_dir, sentences_per_sample=3):
    results = []
    for filename in tqdm.autonotebook.tqdm(os.listdir(data_dir)):
        if not filename.endswith('.txt'):
            continue
        filepath = os.path.join(data_dir, filename)
        if os.path.isfile(filepath):
            file_samples = []
            file_samples_no_punct = []
            with open(filepath) as file:
                sentences = sent_tokenize(file.read())
                for i in range(len(sentences) - sentences_per_sample):
                    sample = ' '.join(sentences[i:i + sentences_per_sample])
                            
                    file_samples.append(prepare(sample))
                    file_samples_no_punct.append(prepare(sample, keep_punct=False))
            if len(file_samples) > 0:
                file_samples_predict = make_punct(file_samples_no_punct, model, tokenizer)
                for sample, sample_pred in zip(file_samples, file_samples_predict):
                    try:
                        results.append(check_one(sample, sample_pred))
                    except Exception as e:
                        print(e)
    return results

In [18]:
results = eval_data('Lenta/test')

In [19]:
import numpy as np
print(np.average(results))

0.8851497053424776


А теперь попробуем евалить, не учитывая, что после слова может быть больше 1 знака пунктуации

In [20]:
def eval_dataset(dataset):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, collate_fn=transformers.default_data_collator)
    model.eval()
    results = []
    for batch in tqdm.autonotebook.tqdm(dataloader):
        torch.cuda.empty_cache()
        batch = {k: t.to('cuda') for k, t in batch.items()}
        with torch.no_grad():
            model_outputs = model(**batch)
            
        labels_true = batch['labels'].cpu()
        labels_predict = torch.argmax(model_outputs[1], dim=-1).cpu()
        
        for sample_ids, sample_true_labels, sample_predict_labels in list(zip(batch['input_ids'].cpu(), labels_true, labels_predict)):
            sample = restore_punct(sample_ids, tokenizer, sample_true_labels)
            sample_pred = restore_punct(sample_ids, tokenizer, sample_predict_labels)
            
            try:
                results.append(check_one(sample, sample_pred))
            except Exception as e:
                print(e)

    return results

In [21]:
results = eval_dataset(dataset_test)

division by zero



In [22]:
import numpy as np
print(np.average(results))

0.9202121713530056
